# AWS Lambda Operations - Comprehensive Guide

This notebook provides a complete walkthrough of AWS Lambda serverless compute operations using boto3 SDK.

## Prerequisites - Manual AWS Setup

Before running this notebook, the following setup was completed manually in AWS Console:

### Step 1: Create IAM Role for Lambda Execution
- Logged into AWS account
- Navigate to **IAM** > **Roles** > **Create role**
- Select trusted entity: **AWS service** > **Lambda**
- Attach policies:
  - **AWSLambdaBasicExecutionRole** (CloudWatch Logs access)
  - **AmazonS3FullAccess** (for S3 integration examples)
- Role name: **lambda-execution-role**
- Copy the Role ARN (needed for creating functions)

### Step 2: IAM User for Lambda Management
- Created IAM user with **programmatic access**
- Attached policies:
  - **AWSLambda_FullAccess** (manage Lambda functions)
  - **IAMReadOnlyAccess** (read roles for function creation)
- Generated access credentials (Access Key ID, Secret Access Key)
- Saved credentials securely in `.env` file

### Step 3: Create Simple Lambda Function (Manual Setup)
- Navigate to **Lambda** > **Create function**
- Function name: **hello-world-function**
- Runtime: **Python 3.12**
- Execution role: Use existing role **lambda-execution-role**
- Created function successfully
- This will be used for testing our boto3 operations

### Step 4: Environment File
Created `.env` file with the following variables:
```
AWS_ACCESS_KEY=your-access-key-id
AWS_SECRET_KEY=your-secret-access-key
AWS_REGION=us-east-2
LAMBDA_ROLE_ARN=arn:aws:iam::YOUR-ACCOUNT-ID:role/lambda-execution-role
```

### Step 5: Python Environment
- Installed boto3 library for AWS SDK
- Installed python-dotenv for secure credential loading

Now we're ready to explore Lambda operations programmatically.

---


## What You'll Learn

### Phase 1: Basic Lambda Operations (Read-Only)
- List all Lambda functions in your account
- Get function details (configuration, memory, timeout)
- Invoke functions and retrieve responses
- View function logs

### Phase 2: Function Management (Create & Deploy)
- Create Lambda functions programmatically
- Upload and update function code
- Delete functions
- Package Python code for deployment

### Phase 3: Configuration & Environment
- Set environment variables
- Configure memory and timeout settings
- Update execution role
- Manage function aliases and versions

### Phase 4: Event Sources & Triggers
- Create S3 event triggers
- Set up scheduled execution (EventBridge/CloudWatch)
- Configure API Gateway integration
- Process event data from different sources

### Phase 5: Advanced Features
- Create and attach Lambda layers
- View CloudWatch logs programmatically
- Implement error handling and retries
- Monitor function metrics
- Optimize performance and cost


## Lambda Architecture Overview

```
┌────────────────────────────────────────────────────────┐
│  AWS LAMBDA ARCHITECTURE                               │
│                                                        │
│  Event Sources                Lambda Function          │
│  ┌──────────────┐            ┌──────────────┐          │
│  │ S3 Upload    │───────────>│              │          │
│  │ API Request  │───────────>│  Your Code   │          │
│  │ Schedule     │───────────>│  (Handler)   │          │
│  │ Manual Invoke│───────────>│              │          │
│  └──────────────┘            └──────┬───────┘          │
│                                     │                  │
│                                     ▼                  │
│  AWS Manages:              ┌──────────────┐            │
│  • Server provisioning     │   Outputs:   │            │
│  • Scaling                 │   • Return   │            │
│  • High availability       │   • S3 Write │            │
│  • Patching                │   • Database │            │
│  • Monitoring              └──────────────┘            │
│                                                        │
│  You Pay Only For:                                     │
│  • Number of requests (invocations)                    │
│  • Compute time (GB-seconds)                           │
└────────────────────────────────────────────────────────┘
```


## Key Concepts

**Function**: Your code package with handler, runtime, and dependencies

**Handler**: Entry point function (e.g., `lambda_handler(event, context)`)

**Runtime**: Execution environment (Python 3.12, Node.js 20, Java 11, etc.)

**Event**: Input data passed to your function

**Context**: Runtime information (request ID, memory limit, etc.)

**Execution Role**: IAM role that grants permissions to your function

**Trigger**: Event source that invokes your function

**Cold Start**: First invocation after deployment (slower)

**Warm Start**: Subsequent invocations (faster, reuses container)

In [27]:
import boto3
from dotenv import load_dotenv
load_dotenv()
import os
import json

## 1. Environment Setup

Loading AWS credentials securely from environment variables using `.env` file.

**Security Best Practice:** Never hardcode credentials in your code!

In [57]:
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")
LAMBDA_ROLE_ARN = os.getenv("LAMBDA_ROLE_ARN")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")

## 2. Initialize Lambda Client

Create a boto3 Lambda client to interact with AWS Lambda service.

In [29]:
lambda_client = boto3.client('lambda',
                              region_name=AWS_REGION,
                              aws_access_key_id=ACCESS_KEY,
                              aws_secret_access_key=SECRET_KEY)

## PHASE 1: BASIC LAMBDA OPERATIONS

Starting with read-only operations to safely explore your Lambda functions.

### Function 1: List All Lambda Functions

Retrieve a list of all Lambda functions in your AWS account.

In [30]:
from botocore.exceptions import ClientError

def list_lambda_functions():
    """
    List all Lambda functions in your AWS account
    
    Returns:
        list: List of function names, or empty list if error
    """
    try:
        response = lambda_client.list_functions()
        
        functions = response.get('Functions', [])
        
        if not functions:
            print("No Lambda functions found in your account")
            return []
        
        print(f"Found {len(functions)} Lambda function(s):")
        
        function_names = []
        for func in functions:
            name = func['FunctionName']
            runtime = func['Runtime']
            memory = func['MemorySize']
            timeout = func['Timeout']
            last_modified = func['LastModified']
            
            print(f"Function: {name}")
            print(f"Runtime: {runtime}")
            print(f"Memory: {memory} MB")
            print(f"Timeout: {timeout} seconds")
            print(f"Last Modified: {last_modified}")
            
            function_names.append(name)
        
        return function_names
        
    except ClientError as e:
        print(f"ERROR: Failed to list functions - {e}")
        return []

# Test the function
function_names = list_lambda_functions()

Found 2 Lambda function(s):
Function: boto3-hello-world
Runtime: python3.12
Memory: 256 MB
Timeout: 30 seconds
Last Modified: 2026-01-30T03:00:54.000+0000
Function: real_aws_lambda
Runtime: python3.12
Memory: 128 MB
Timeout: 3 seconds
Last Modified: 2026-01-29T07:50:16.061+0000


### Function 2: Get Function Configuration

Retrieve detailed configuration for a specific Lambda function including code size, environment variables, and execution role.

In [31]:
import re

def redact_account_id(arn):
    """Redact AWS account ID from ARN for security"""
    return re.sub(r':\d{12}:', ':************:', arn)

def get_function_config(function_name):
    """
    Get detailed configuration for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
    
    Returns:
        dict: Function configuration, or None if error
    """
    try:
        response = lambda_client.get_function_configuration(
            FunctionName=function_name
        )
        
        print(f"Configuration for '{function_name}':")
        
        # Basic info (redact account ID from ARN)
        print(f"Function ARN: {redact_account_id(response['FunctionArn'])}")
        print(f"Runtime: {response['Runtime']}")
        print(f"Handler: {response['Handler']}")
        print(f"Memory Size: {response['MemorySize']} MB")
        print(f"Timeout: {response['Timeout']} seconds")
        print(f"Code Size: {response['CodeSize']:,} bytes")
        
        # Execution role (redact account ID)
        print(f"Execution Role: {redact_account_id(response['Role'])}")
        
        # Environment variables (show keys only, hide values)
        env_vars = response.get('Environment', {}).get('Variables', {})
        if env_vars:
            print(f"Environment Variables:")
            for key in env_vars.keys():
                print(f"  {key}: [REDACTED]")
        else:
            print(f"Environment Variables: None")
        
        # Layers (redact account ID)
        layers = response.get('Layers', [])
        if layers:
            print(f"Layers:")
            for layer in layers:
                print(f"  - {redact_account_id(layer['Arn'])}")
        else:
            print(f"Layers: None")
        
        # VPC config
        vpc_config = response.get('VpcConfig', {})
        if vpc_config.get('VpcId'):
            print(f"VPC ID: {vpc_config['VpcId']}")
        else:
            print(f"VPC: Not configured")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to get function config - {e}")
        return None

# Test with the first function from our list
if function_names:
    config = get_function_config(function_names[0])

Configuration for 'boto3-hello-world':
Function ARN: arn:aws:lambda:us-east-2:************:function:boto3-hello-world
Runtime: python3.12
Handler: lambda_function.lambda_handler
Memory Size: 256 MB
Timeout: 30 seconds
Code Size: 387 bytes
Execution Role: arn:aws:iam::************:role/real-aws-lambda-s3-access
Environment Variables:
  ENVIRONMENT: [REDACTED]
  FEATURE_FLAG: [REDACTED]
  LOG_LEVEL: [REDACTED]
Layers: None
VPC: Not configured


### Function 3: Invoke Lambda Function

Execute a Lambda function and retrieve its response. This is the core operation for running serverless code.

In [32]:
def invoke_lambda(function_name, payload=None, invocation_type='RequestResponse'):
    """
    Invoke a Lambda function with optional payload
    
    Args:
        function_name (str): Name of the Lambda function
        payload (dict): Input data for the function (optional)
        invocation_type (str): 
            - 'RequestResponse' (default): Synchronous, wait for response
            - 'Event': Asynchronous, don't wait for response
            - 'DryRun': Validate parameters without executing
    
    Returns:
        dict: Function response including status code and payload
    """
    try:
        # Prepare payload
        if payload is None:
            payload = {}
        
        # Convert payload to JSON bytes
        payload_bytes = json.dumps(payload).encode('utf-8')
        
        print(f"Invoking '{function_name}' with invocation type: {invocation_type}")
        print(f"Payload: {json.dumps(payload, indent=2)}")
        print("-" * 80)
        
        # Invoke function
        response = lambda_client.invoke(
            FunctionName=function_name,
            InvocationType=invocation_type,
            Payload=payload_bytes
        )
        
        # Extract response details
        status_code = response['StatusCode']
        print(f"Status Code: {status_code}")
        
        # Read response payload
        if invocation_type == 'RequestResponse':
            response_payload = response['Payload'].read().decode('utf-8')
            response_data = json.loads(response_payload)
            
            print(f"Response Payload:")
            print(json.dumps(response_data, indent=2))
            
            # Check for function errors
            if 'FunctionError' in response:
                print(f"Function Error: {response['FunctionError']}")
            else:
                print(f"SUCCESS: Function executed successfully")
            
            return {
                'statusCode': status_code,
                'payload': response_data,
                'executedVersion': response.get('ExecutedVersion', '$LATEST')
            }
        else:
            print(f"Asynchronous invocation initiated")
            return {
                'statusCode': status_code,
                'message': 'Asynchronous invocation - no immediate response'
            }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to invoke function - {e}")
        return None

# Test with a simple payload
if function_names:
    test_payload = {
        "name": "Lambda Learner",
        "message": "Testing from boto3"
    }
    result = invoke_lambda(function_names[0], test_payload)

Invoking 'boto3-hello-world' with invocation type: RequestResponse
Payload: {
  "name": "Lambda Learner",
  "message": "Testing from boto3"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "{\"message\": \"Hello, Lambda Learner! Created via boto3.\", \"requestId\": \"a4f8faf8-f8aa-4bdc-8d53-b97c30e5b6fe\"}"
}
SUCCESS: Function executed successfully


## PHASE 2: FUNCTION MANAGEMENT

Now we'll learn to create, update, and delete Lambda functions programmatically.

### Function 4: Create Lambda Function

Create a new Lambda function from Python code.

In [33]:
import zipfile
import io

def create_lambda_function(function_name, handler_code, handler='lambda_function.lambda_handler', 
                           runtime='python3.12', memory=128, timeout=30, description=''):
    """
    Create a new Lambda function with Python code
    
    Args:
        function_name (str): Name for the new function
        handler_code (str): Python code for the Lambda handler
        handler (str): Handler function path (default: lambda_function.lambda_handler)
        runtime (str): Python runtime version (default: python3.12)
        memory (int): Memory in MB (128-10240, default: 128)
        timeout (int): Timeout in seconds (1-900, default: 30)
        description (str): Function description
    
    Returns:
        dict: Created function configuration, or None if error
    
    Example handler_code:
        '''
        import json
        def lambda_handler(event, context):
            return {
                'statusCode': 200,
                'body': json.dumps('Hello from Lambda!')
            }
        '''
    """
    try:
        # Create in-memory ZIP file
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Add handler code to ZIP
            zip_file.writestr('lambda_function.py', handler_code)
        
        # Get ZIP bytes
        zip_bytes = zip_buffer.getvalue()
        
        print(f"Creating Lambda function '{function_name}'...")
        print(f"Runtime: {runtime}")
        print(f"Memory: {memory} MB")
        print(f"Timeout: {timeout} seconds")
        print(f"Code Size: {len(zip_bytes):,} bytes")
        
        # Create function
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime=runtime,
            Role=LAMBDA_ROLE_ARN,
            Handler=handler,
            Code={'ZipFile': zip_bytes},
            Description=description,
            Timeout=timeout,
            MemorySize=memory,
            Publish=True  # Publish initial version
        )
        
        print(f"SUCCESS: Function '{function_name}' created successfully")
        print(f"Function ARN: {redact_account_id(response['FunctionArn'])}")
        print(f"Version: {response['Version']}")
        print(f"State: {response['State']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceConflictException':
            print(f"ERROR: Function '{function_name}' already exists")
        elif error_code == 'InvalidParameterValueException':
            print(f"ERROR: Invalid parameter - {e.response['Error']['Message']}")
        else:
            print(f"ERROR: Failed to create function - {e}")
        return None

# Test: Create a simple "Hello World" Lambda function
hello_world_code = """
import json

def lambda_handler(event, context):
    \"\"\"
    Simple Hello World Lambda function
    \"\"\"
    # Get name from event or use default
    name = event.get('name', 'World')
    
    # Create response
    message = f"Hello, {name}! Created via boto3."
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': message,
            'requestId': context.aws_request_id
        })
    }
"""

# Create the function
new_function = create_lambda_function(
    function_name='boto3-hello-world',
    handler_code=hello_world_code,
    description='Hello World function created via boto3',
    memory=128,
    timeout=10
)

Creating Lambda function 'boto3-hello-world'...
Runtime: python3.12
Memory: 128 MB
Timeout: 10 seconds
Code Size: 387 bytes
ERROR: Function 'boto3-hello-world' already exists


In [53]:
# create a function to move content file from trigger folder
move_content_file_from_trigger_folder = """
import json
import boto3
import urllib.parse

def lambda_handler(event, context):
    \"\"\"
    Moves files from trigger folder to destination folder in S3.
    Triggered by S3 ObjectCreated events.
    \"\"\"
    s3_client = boto3.client('s3')
    
    # Process each record in the event
    for record in event.get('Records', []):
        # Get bucket and object info from the S3 event
        bucket = record['s3']['bucket']['name']
        source_key = urllib.parse.unquote_plus(record['s3']['object']['key'])
        
        # Define source and destination folders
        source_folder = 'trigger/'
        dest_folder = 'processed/'
        
        # Skip if not in the source folder
        if not source_key.startswith(source_folder):
            print(f"Skipping {source_key} - not in {source_folder}")
            continue
        
        # Build destination key (replace source folder with dest folder)
        filename = source_key[len(source_folder):]
        dest_key = dest_folder + filename
        
        print(f"Moving {source_key} -> {dest_key}")
        
        try:
            # Copy to destination
            s3_client.copy_object(
                Bucket=bucket,
                CopySource={'Bucket': bucket, 'Key': source_key},
                Key=dest_key
            )
            
            # Delete from source
            s3_client.delete_object(Bucket=bucket, Key=source_key)
            
            print(f"SUCCESS: Moved {filename} to {dest_folder}")
            
        except Exception as e:
            print(f"ERROR: Failed to move {source_key} - {str(e)}")
            raise
    
    return {
        'statusCode': 200,
        'body': json.dumps({'message': 'Files processed successfully'})
    }
"""

# Create the function
new_function = create_lambda_function(
    function_name='move-content-file-from-trigger-folder',
    handler_code=move_content_file_from_trigger_folder,
    description='Moves files from trigger/ to processed/ folder on S3 upload',
    memory=128,
    timeout=30
)


Creating Lambda function 'move-content-file-from-trigger-folder'...
Runtime: python3.12
Memory: 128 MB
Timeout: 30 seconds
Code Size: 817 bytes
SUCCESS: Function 'move-content-file-from-trigger-folder' created successfully
Function ARN: arn:aws:lambda:us-east-2:************:function:move-content-file-from-trigger-folder
Version: 1
State: Pending


In [34]:
# Test the newly created function
print("Testing the new function:")
invoke_lambda('boto3-hello-world', {'name': 'Student'})

Testing the new function:
Invoking 'boto3-hello-world' with invocation type: RequestResponse
Payload: {
  "name": "Student"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "{\"message\": \"Hello, Student! Created via boto3.\", \"requestId\": \"83ec63b3-1c7a-468a-a8c7-b17f94305736\"}"
}
SUCCESS: Function executed successfully


{'statusCode': 200,
 'payload': {'statusCode': 200,
  'body': '{"message": "Hello, Student! Created via boto3.", "requestId": "83ec63b3-1c7a-468a-a8c7-b17f94305736"}'},
 'executedVersion': '$LATEST'}

### Function 5: Update Lambda Function Code

Update the code of an existing Lambda function.

In [35]:
def update_lambda_code(function_name, handler_code):
    """
    Update the code for an existing Lambda function
    
    Args:
        function_name (str): Name of the function to update
        handler_code (str): New Python code for the handler
    
    Returns:
        dict: Updated function configuration, or None if error
    """
    try:
        # Create in-memory ZIP file with new code
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.writestr('lambda_function.py', handler_code)
        
        zip_bytes = zip_buffer.getvalue()
        
        print(f"Updating code for '{function_name}'...")
        print(f"New code size: {len(zip_bytes):,} bytes")
        print("-" * 80)
        
        # Update function code
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_bytes,
            Publish=True  # Publish new version
        )
        
        print(f"SUCCESS: Function '{function_name}' code updated")
        print(f"New Version: {response['Version']}")
        print(f"Code Size: {response['CodeSize']:,} bytes")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to update function code - {e}")
        return None

# Test: Update the hello-world function with enhanced functionality
updated_code = """
import json
from datetime import datetime

def lambda_handler(event, context):
    \"\"\"
    Enhanced Hello World with timestamp
    \"\"\"
    name = event.get('name', 'World')
    timestamp = datetime.utcnow().isoformat()
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f"Hello, {name}! (Updated version)",
            'timestamp': timestamp,
            'requestId': context.aws_request_id,
            'version': 'v2.0'
        })
    }
"""

# Update the function
updated = update_lambda_code('boto3-hello-world', updated_code)

Updating code for 'boto3-hello-world'...
New code size: 401 bytes
--------------------------------------------------------------------------------
SUCCESS: Function 'boto3-hello-world' code updated
New Version: 7
Code Size: 401 bytes
Last Modified: 2026-01-30T03:07:48.000+0000


In [36]:
# Test the updated function
if updated:
    print("Testing updated function:")
    invoke_lambda('boto3-hello-world', {'name': 'Updated Test'})

Testing updated function:
Invoking 'boto3-hello-world' with invocation type: RequestResponse
Payload: {
  "name": "Updated Test"
}
--------------------------------------------------------------------------------
Status Code: 200
Response Payload:
{
  "statusCode": 200,
  "body": "{\"message\": \"Hello, Updated Test! Created via boto3.\", \"requestId\": \"6b40ddcb-d095-41c6-a261-3020bcc3671c\"}"
}
SUCCESS: Function executed successfully


### Function 6: Delete Lambda Function

Delete a Lambda function. Use with caution - this operation is permanent.

In [37]:
def delete_lambda_function(function_name):
    """
    Delete a Lambda function
    
    Args:
        function_name (str): Name of the function to delete
    
    Returns:
        bool: True if deleted successfully, False otherwise
    
    WARNING: This operation is permanent and cannot be undone!
    """
    try:
        print(f"WARNING: Deleting function '{function_name}'...")
        print("This operation is PERMANENT and cannot be undone!")
        
        # Delete function
        lambda_client.delete_function(FunctionName=function_name)
        
        print(f"SUCCESS: Function '{function_name}' deleted successfully")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found (already deleted?)")
        else:
            print(f"ERROR: Failed to delete function - {e}")
        return False

# Example: Delete the test function we created
# UNCOMMENT TO TEST (be careful - this is permanent):
delete_lambda_function('boto3-hello-world')

# Verify deletion by listing functions
list_lambda_functions()

This operation is PERMANENT and cannot be undone!
SUCCESS: Function 'boto3-hello-world' deleted successfully
Found 1 Lambda function(s):
Function: real_aws_lambda
Runtime: python3.12
Memory: 128 MB
Timeout: 3 seconds
Last Modified: 2026-01-29T07:50:16.061+0000


['real_aws_lambda']

In [38]:
# after deleting boto3-hello-world, lets create it again to verify everything works
create_lambda_function(
    function_name='boto3-hello-world',
    handler_code=hello_world_code,
    description='Hello World function created via boto3',
    memory=128,
    timeout=10
)

Creating Lambda function 'boto3-hello-world'...
Runtime: python3.12
Memory: 128 MB
Timeout: 10 seconds
Code Size: 387 bytes
SUCCESS: Function 'boto3-hello-world' created successfully
Function ARN: arn:aws:lambda:us-east-2:************:function:boto3-hello-world
Version: 8
State: Pending


{'ResponseMetadata': {'RequestId': '21f24adb-77e0-47e7-9315-ec45e01b09c8',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Fri, 30 Jan 2026 03:07:49 GMT',
   'content-type': 'application/json',
   'content-length': '1472',
   'connection': 'keep-alive',
   'x-amzn-requestid': '21f24adb-77e0-47e7-9315-ec45e01b09c8'},
  'RetryAttempts': 0},
 'FunctionName': 'boto3-hello-world',
 'FunctionArn': 'arn:aws:lambda:us-east-2:445952351133:function:boto3-hello-world',
 'Runtime': 'python3.12',
 'Role': 'arn:aws:iam::445952351133:role/real-aws-lambda-s3-access',
 'Handler': 'lambda_function.lambda_handler',
 'CodeSize': 387,
 'Description': 'Hello World function created via boto3',
 'Timeout': 10,
 'MemorySize': 128,
 'LastModified': '2026-01-30T03:07:48.819+0000',
 'CodeSha256': '5XRrl8e2ZtdRDmQK1vUlr9zunEHYojDFMb3zUfa7Ob0=',
 'Version': '8',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': 'b713ac9c-e619-4a43-9e3a-2f6fe99ed26e',
 'State': 'Pending',
 'StateReason': 'The function is

## PHASE 3: CONFIGURATION & ENVIRONMENT

Now we'll learn to configure Lambda functions by setting environment variables, adjusting memory and timeout, and managing function aliases and versions.

### Function 7: Set Environment Variables

Configure environment variables for a Lambda function. These are accessible within your function code via `os.environ`.

In [48]:
def set_environment_variables(function_name, env_vars):
    """
    Set or update environment variables for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        env_vars (dict): Dictionary of environment variables (key-value pairs)
    
    Returns:
        dict: Updated function configuration, or None if error
    
    Example:
        set_environment_variables('my-function', {
            'DATABASE_URL': 'postgresql://...',
            'API_KEY': 'secret-key',
            'DEBUG': 'true'
        })
    """
    try:
        print(f"Setting environment variables for '{function_name}'...")
        print(f"Variables to set:")
        for key, value in env_vars.items():
            # Mask sensitive values in output
            display_value = value if len(value) < 20 else f"{value[:10]}...{value[-5:]}"
            print(f"  {key}: {display_value}")
        
        # Update function configuration
        response = lambda_client.update_function_configuration(
            FunctionName=function_name,
            Environment={
                'Variables': env_vars
            }
        )
        
        print(f"SUCCESS: Environment variables updated")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to set environment variables - {e}")
        return None

# Test: Add environment variables to our test function
test_env = {
    'ENVIRONMENT': 'development',
    'LOG_LEVEL': 'INFO',
    'FEATURE_FLAG': 'enabled'
}

env_result = set_environment_variables('boto3-hello-world', test_env)

Setting environment variables for 'boto3-hello-world'...
Variables to set:
  ENVIRONMENT: development
  LOG_LEVEL: INFO
  FEATURE_FLAG: enabled
SUCCESS: Environment variables updated
Last Modified: 2026-01-30T03:08:51.000+0000


### Function 8: Update Function Configuration

Update memory allocation and timeout settings for a Lambda function to optimize performance and cost.

In [49]:
def update_function_configuration(function_name, memory=None, timeout=None, description=None):
    """
    Update configuration settings for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        memory (int): Memory in MB (128-10240, multiples of 64)
        timeout (int): Timeout in seconds (1-900)
        description (str): Function description
    
    Returns:
        dict: Updated function configuration, or None if error
    
    Note: 
        - More memory = more CPU power (proportional)
        - Memory affects cost: $0.0000166667 per GB-second
        - Timeout max is 15 minutes (900 seconds)
    """
    try:
        # Build update parameters
        update_params = {'FunctionName': function_name}
        
        if memory is not None:
            if memory < 128 or memory > 10240:
                print(f"ERROR: Memory must be between 128 and 10240 MB")
                return None
            update_params['MemorySize'] = memory
        
        if timeout is not None:
            if timeout < 1 or timeout > 900:
                print(f"ERROR: Timeout must be between 1 and 900 seconds")
                return None
            update_params['Timeout'] = timeout
        
        if description is not None:
            update_params['Description'] = description
        
        if len(update_params) == 1:
            print("ERROR: No configuration changes specified")
            return None
        
        print(f"Updating configuration for '{function_name}'...")
        if memory:
            print(f"Memory: {memory} MB")
        if timeout:
            print(f"Timeout: {timeout} seconds")
        if description:
            print(f"Description: {description}")
        
        # Update function configuration
        response = lambda_client.update_function_configuration(**update_params)
        
        print(f"SUCCESS: Configuration updated")
        print(f"Current Memory: {response['MemorySize']} MB")
        print(f"Current Timeout: {response['Timeout']} seconds")
        print(f"Last Modified: {response['LastModified']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to update configuration - {e}")
        return None

# Test: Update memory and timeout for better performance
config_result = update_function_configuration(
    'boto3-hello-world',
    memory=256,  # Increase from 128 to 256 MB
    timeout=30,  # Increase from 10 to 30 seconds
    description='Hello World function - updated configuration'
)

Updating configuration for 'boto3-hello-world'...
Memory: 256 MB
Timeout: 30 seconds
Description: Hello World function - updated configuration
SUCCESS: Configuration updated
Current Memory: 256 MB
Current Timeout: 30 seconds
Last Modified: 2026-01-30T03:08:57.000+0000


### Function 9: Create Function Alias

Create an alias for a Lambda function version. Aliases are pointers to specific versions and can be used for traffic splitting and blue/green deployments.

In [50]:
def create_function_alias(function_name, alias_name, version, description=''):
    """
    Create an alias pointing to a specific function version
    
    Args:
        function_name (str): Name of the Lambda function
        alias_name (str): Name for the alias (e.g., 'prod', 'staging', 'dev')
        version (str): Version number to point to (or '$LATEST')
        description (str): Alias description
    
    Returns:
        dict: Created alias configuration, or None if error
    
    Use cases:
        - Blue/Green deployments: Switch traffic between versions
        - Environment separation: prod, staging, dev aliases
        - Traffic splitting: Route percentage to different versions
    """
    try:
        print(f"Creating alias '{alias_name}' for '{function_name}'...")
        print(f"Points to version: {version}")
        
        # Create alias
        response = lambda_client.create_alias(
            FunctionName=function_name,
            Name=alias_name,
            FunctionVersion=version,
            Description=description
        )
        
        print(f"SUCCESS: Alias '{alias_name}' created")
        print(f"Alias ARN: {redact_account_id(response['AliasArn'])}")
        print(f"Function Version: {response['FunctionVersion']}")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceConflictException':
            print(f"ERROR: Alias '{alias_name}' already exists")
        elif error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' or version '{version}' not found")
        else:
            print(f"ERROR: Failed to create alias - {e}")
        return None

# Test: Create a 'prod' alias pointing to version 4
alias_result = create_function_alias(
    'boto3-hello-world',
    alias_name='prod',
    version='2',
    description='Production version'
)

Creating alias 'prod' for 'boto3-hello-world'...
Points to version: 2
ERROR: Function 'boto3-hello-world' or version '2' not found


### Function 10: List Function Versions

List all versions of a Lambda function to track deployment history.

In [51]:
def list_function_versions(function_name):
    """
    List all versions of a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
    
    Returns:
        list: List of version configurations, or empty list if error
    
    Note: 
        - $LATEST is always available (unpublished changes)
        - Published versions are immutable
        - Version numbers are sequential (1, 2, 3, ...)
    """
    try:
        print(f"Listing versions for '{function_name}'...")
        print("-" * 80)
        
        # List all versions
        response = lambda_client.list_versions_by_function(
            FunctionName=function_name
        )
        
        versions = response.get('Versions', [])
        
        if not versions:
            print(f"No versions found for '{function_name}'")
            return []
        
        print(f"Found {len(versions)} version(s):")
        
        version_list = []
        for version in versions:
            version_num = version['Version']
            code_size = version['CodeSize']
            last_modified = version['LastModified']
            description = version.get('Description', 'No description')
            
            print(f"Version: {version_num}")
            print(f"Code Size: {code_size:,} bytes")
            print(f"Last Modified: {last_modified}")
            print(f"Description: {description}")
            
            version_list.append({
                'version': version_num,
                'codeSize': code_size,
                'lastModified': last_modified,
                'description': description
            })
        
        return version_list
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to list versions - {e}")
        return []

# Test: List all versions of our test function
versions = list_function_versions('boto3-hello-world')

Listing versions for 'boto3-hello-world'...
--------------------------------------------------------------------------------
Found 2 version(s):
Version: $LATEST
Code Size: 387 bytes
Last Modified: 2026-01-30T03:08:57.000+0000
Description: Hello World function - updated configuration
Version: 8
Code Size: 387 bytes
Last Modified: 2026-01-30T03:07:48.819+0000
Description: Hello World function created via boto3


## PHASE 4: EVENT SOURCES & TRIGGERS

Now we'll learn to configure event sources that automatically trigger Lambda functions, including S3 uploads, scheduled events, and more.

### Function 11: Add S3 Event Trigger

Configure an S3 bucket to trigger a Lambda function when objects are uploaded.

In [54]:
def add_s3_trigger(function_name, bucket_name, events=['s3:ObjectCreated:*'], prefix='', suffix=''):
    """
    Add S3 bucket notification to trigger Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        bucket_name (str): Name of the S3 bucket
        events (list): S3 events to trigger on (default: all object creations)
        prefix (str): Filter by object key prefix (e.g., 'uploads/')
        suffix (str): Filter by object key suffix (e.g., '.jpg')
    
    Returns:
        dict: Notification configuration, or None if error
    
    Common S3 events:
        - s3:ObjectCreated:* - Any object creation
        - s3:ObjectCreated:Put - Object uploaded via PUT
        - s3:ObjectCreated:Post - Object uploaded via POST
        - s3:ObjectRemoved:* - Any object deletion
    
    Note: Requires Lambda permission for S3 to invoke the function
    """
    try:
        # First, add permission for S3 to invoke Lambda
        print(f"Adding S3 invoke permission for '{function_name}'...")
        
        try:
            lambda_client.add_permission(
                FunctionName=function_name,
                StatementId=f's3-invoke-{bucket_name}',
                Action='lambda:InvokeFunction',
                Principal='s3.amazonaws.com',
                SourceArn=f'arn:aws:s3:::{bucket_name}'
            )
            print(f"Permission added successfully")
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceConflictException':
                print(f"Permission already exists (OK)")
            else:
                raise
        
        # Get function ARN
        func_config = lambda_client.get_function_configuration(FunctionName=function_name)
        function_arn = func_config['FunctionArn']
        
        # Create S3 client
        s3_client = boto3.client('s3',
                                region_name=AWS_REGION,
                                aws_access_key_id=ACCESS_KEY,
                                aws_secret_access_key=SECRET_KEY)
        
        # Build notification configuration
        notification_config = {
            'LambdaFunctionConfigurations': [
                {
                    'LambdaFunctionArn': function_arn,
                    'Events': events
                }
            ]
        }
        
        # Add filters if specified
        if prefix or suffix:
            filter_rules = []
            if prefix:
                filter_rules.append({'Name': 'prefix', 'Value': prefix})
            if suffix:
                filter_rules.append({'Name': 'suffix', 'Value': suffix})
            
            notification_config['LambdaFunctionConfigurations'][0]['Filter'] = {
                'Key': {'FilterRules': filter_rules}
            }
        
        # Apply notification configuration
        print(f"Configuring S3 trigger on bucket '{bucket_name}'...")
        print(f"Events: {', '.join(events)}")
        if prefix:
            print(f"Prefix filter: {prefix}")
        if suffix:
            print(f"Suffix filter: {suffix}")
        
        s3_client.put_bucket_notification_configuration(
            Bucket=bucket_name,
            NotificationConfiguration=notification_config
        )
        
        print(f"SUCCESS: S3 trigger configured")
        print(f"Lambda function '{function_name}' will be triggered on {', '.join(events)}")
        
        return notification_config
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'NoSuchBucket':
            print(f"ERROR: Bucket '{bucket_name}' not found")
        else:
            # Redact any ARNs that might appear in error messages
            error_msg = str(e)
            error_msg = redact_account_id(error_msg)
            print(f"ERROR: Failed to add S3 trigger - {error_msg}")
        return None

In [56]:

# Test: Add S3 trigger for uploads to our bucket
# NOTE: This will trigger the function when files are uploaded to real-learn-s3
s3_trigger = add_s3_trigger(
    'boto3-hello-world',
    bucket_name=os.getenv('AWS_BUCKET_NAME'),
    events=['s3:ObjectCreated:*'],
    prefix='lambda-test/'  # Only trigger for files in lambda-test/ folder
)

Adding S3 invoke permission for 'boto3-hello-world'...
Permission added successfully
Configuring S3 trigger on bucket 'real-learn-s3'...
Events: s3:ObjectCreated:*
Prefix filter: lambda-test/
SUCCESS: S3 trigger configured
Lambda function 'boto3-hello-world' will be triggered on s3:ObjectCreated:*


In [59]:
# Test: Add S3 trigger for uploads to our bucket
# NOTE: This will trigger the function when files are uploaded to real-learn-s3
s3_trigger = add_s3_trigger(
    'move-content-file-from-trigger-folder',
    bucket_name=os.getenv('AWS_BUCKET_NAME'),
    events=['s3:ObjectCreated:*'],
    prefix='trigger/'  # Only trigger for files in trigger/ folder
)

Adding S3 invoke permission for 'move-content-file-from-trigger-folder'...
Permission already exists (OK)
Configuring S3 trigger on bucket 'real-learn-s3'...
Events: s3:ObjectCreated:*
Prefix filter: trigger/
SUCCESS: S3 trigger configured
Lambda function 'move-content-file-from-trigger-folder' will be triggered on s3:ObjectCreated:*


We can test the trigger `move-content-file-from-trigger-folder` by calling our upload function to upload a file to the `trigger` folder

In [63]:
# test the S3 trigger `move-content-file-from-trigger-folder` by uploading a file to the 'trigger/' folder in the S3 bucket

# create an s3 client
from botocore.config import Config

# Configure S3 client with Signature Version 4 and regional endpoint
# Regional endpoint is required for presigned URLs to work correctly

s3_client = boto3.client('s3', 
                         endpoint_url=f'https://s3.{AWS_REGION}.amazonaws.com',
                         config=Config(signature_version='s3v4'),
                         region_name=AWS_REGION,
                         aws_secret_access_key=SECRET_KEY,
                         aws_access_key_id=ACCESS_KEY,
                         )

def upload_file(file_name, bucket, object_name=None):
    """
    Upload a file to an S3 bucket
    
    Args:
        file_name (str): Path to file to upload
        bucket (str): Bucket name
        object_name (str): S3 object name (if None, uses file_name)
    
    Returns:
        bool: True if upload successful, False otherwise
    """
    # If S3 object_name not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)
    
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"SUCCESS: '{file_name}' uploaded to '{bucket}/{object_name}'")
        return True
    except FileNotFoundError:
        print(f"ERROR: File '{file_name}' not found")
        return False
    except ClientError as e:
        print(f"ERROR: Failed to upload file - {e}")
        return False

# Example usage (uncomment to test):
upload_file('data/bookings.csv', BUCKET_NAME, 'trigger/bookings.csv')


SUCCESS: 'data/bookings.csv' uploaded to 'real-learn-s3/trigger/bookings.csv'


True

### Function 12: Create Scheduled Event (EventBridge Rule)

Schedule a Lambda function to run at specific times using EventBridge (formerly CloudWatch Events).

In [44]:
def create_scheduled_event(function_name, rule_name, schedule_expression, description=''):
    """
    Create EventBridge rule to trigger Lambda on a schedule
    
    Args:
        function_name (str): Name of the Lambda function
        rule_name (str): Name for the EventBridge rule
        schedule_expression (str): Cron or rate expression
        description (str): Rule description
    
    Returns:
        dict: Rule configuration, or None if error
    
    Schedule expression examples:
        - Rate: 'rate(5 minutes)' - Every 5 minutes
        - Rate: 'rate(1 hour)' - Every hour
        - Rate: 'rate(1 day)' - Every day
        - Cron: 'cron(0 12 * * ? *)' - Every day at 12:00 PM UTC
        - Cron: 'cron(0 18 ? * MON-FRI *)' - Weekdays at 6:00 PM UTC
        - Cron: 'cron(0 0 1 * ? *)' - First day of every month at midnight
    
    Cron format: minute hour day-of-month month day-of-week year
    """
    try:
        # Create EventBridge client
        events_client = boto3.client('events',
                                     region_name=AWS_REGION,
                                     aws_access_key_id=ACCESS_KEY,
                                     aws_secret_access_key=SECRET_KEY)
        
        print(f"Creating scheduled event rule '{rule_name}'...")
        print(f"Schedule: {schedule_expression}")
        
        # Create the rule
        rule_response = events_client.put_rule(
            Name=rule_name,
            ScheduleExpression=schedule_expression,
            State='ENABLED',
            Description=description
        )
        
        rule_arn = rule_response['RuleArn']
        print(f"Rule created: {redact_account_id(rule_arn)}")
        
        # Get function ARN
        func_config = lambda_client.get_function_configuration(FunctionName=function_name)
        function_arn = func_config['FunctionArn']
        
        # Add permission for EventBridge to invoke Lambda
        print(f"Adding EventBridge invoke permission...")
        try:
            lambda_client.add_permission(
                FunctionName=function_name,
                StatementId=f'eventbridge-invoke-{rule_name}',
                Action='lambda:InvokeFunction',
                Principal='events.amazonaws.com',
                SourceArn=rule_arn
            )
            print(f"Permission added successfully")
        except ClientError as e:
            if e.response['Error']['Code'] == 'ResourceConflictException':
                print(f"Permission already exists (OK)")
            else:
                raise
        
        # Add Lambda as target for the rule
        print(f"Adding Lambda function as target...")
        events_client.put_targets(
            Rule=rule_name,
            Targets=[
                {
                    'Id': '1',
                    'Arn': function_arn
                }
            ]
        )
        
        print(f"SUCCESS: Scheduled event configured")
        print(f"Lambda function '{function_name}' will run on schedule: {schedule_expression}")
        
        return {
            'ruleArn': rule_arn,
            'ruleName': rule_name,
            'schedule': schedule_expression,
            'functionArn': function_arn
        }
        
    except ClientError as e:
        print(f"ERROR: Failed to create scheduled event - {e}")
        return None

# Test: Schedule function to run every 5 minutes
# NOTE: Comment out or delete the rule after testing to avoid unnecessary invocations
scheduled_event = create_scheduled_event(
    'boto3-hello-world',
    rule_name='boto3-hello-world-schedule',
    schedule_expression='rate(5 minutes)',
    description='Test schedule - runs every 5 minutes'
)

Creating scheduled event rule 'boto3-hello-world-schedule'...
Schedule: rate(5 minutes)
Rule created: arn:aws:events:us-east-2:************:rule/boto3-hello-world-schedule
Adding EventBridge invoke permission...
Permission added successfully
Adding Lambda function as target...
SUCCESS: Scheduled event configured
Lambda function 'boto3-hello-world' will run on schedule: rate(5 minutes)


### Function 13: Remove Event Source

Remove an event source trigger or scheduled rule from a Lambda function.

In [45]:
def remove_eventbridge_rule(rule_name):
    """
    Remove an EventBridge scheduled rule
    
    Args:
        rule_name (str): Name of the EventBridge rule to remove
    
    Returns:
        bool: True if removed successfully, False otherwise
    """
    try:
        # Create EventBridge client
        events_client = boto3.client('events',
                                     region_name=AWS_REGION,
                                     aws_access_key_id=ACCESS_KEY,
                                     aws_secret_access_key=SECRET_KEY)
        
        print(f"Removing EventBridge rule '{rule_name}'...")
        
        # First, remove all targets from the rule
        print(f"Removing targets from rule...")
        targets_response = events_client.list_targets_by_rule(Rule=rule_name)
        target_ids = [target['Id'] for target in targets_response.get('Targets', [])]
        
        if target_ids:
            events_client.remove_targets(
                Rule=rule_name,
                Ids=target_ids
            )
            print(f"Removed {len(target_ids)} target(s)")
        
        # Then, delete the rule
        events_client.delete_rule(Name=rule_name)
        
        print(f"SUCCESS: EventBridge rule '{rule_name}' removed")
        
        return True
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Rule '{rule_name}' not found")
        else:
            print(f"ERROR: Failed to remove rule - {e}")
        return False

# Test: Remove the scheduled event we just created
# UNCOMMENT TO TEST:
remove_eventbridge_rule('boto3-hello-world-schedule')

Removing EventBridge rule 'boto3-hello-world-schedule'...
Removing targets from rule...
Removed 1 target(s)
SUCCESS: EventBridge rule 'boto3-hello-world-schedule' removed


True

## PHASE 5: MONITORING & DEBUGGING (Data Engineering Essentials)

Essential tools for debugging data pipelines and monitoring Lambda function performance.

### Function 14: Get CloudWatch Logs

Retrieve recent execution logs to debug data processing issues.

In [46]:
from datetime import datetime, timedelta

def get_cloudwatch_logs(function_name, minutes=10, max_events=50):
    """
    Retrieve recent CloudWatch logs for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        minutes (int): Look back this many minutes (default: 10)
        max_events (int): Maximum number of log events to retrieve (default: 50)
    
    Returns:
        list: List of log events with timestamps and messages
    
    Use cases for data engineers:
        - Debug failed data processing jobs
        - View error messages from S3 file processing
        - Check data transformation outputs
        - Verify function execution flow
    """
    try:
        # Create CloudWatch Logs client
        logs_client = boto3.client('logs',
                                   region_name=AWS_REGION,
                                   aws_access_key_id=ACCESS_KEY,
                                   aws_secret_access_key=SECRET_KEY)
        
        # Log group name follows pattern: /aws/lambda/{function_name}
        log_group = f'/aws/lambda/{function_name}'
        
        print(f"Retrieving logs for '{function_name}' (last {minutes} minutes)...")
        print(f"Log Group: {log_group}")
        print("-" * 80)
        
        # Calculate time range
        end_time = datetime.now()
        start_time = end_time - timedelta(minutes=minutes)
        
        # Convert to milliseconds since epoch
        start_ms = int(start_time.timestamp() * 1000)
        end_ms = int(end_time.timestamp() * 1000)
        
        try:
            # Get log streams (recent first)
            streams_response = logs_client.describe_log_streams(
                logGroupName=log_group,
                orderBy='LastEventTime',
                descending=True,
                limit=5  # Check last 5 streams
            )
            
            log_streams = streams_response.get('logStreams', [])
            
            if not log_streams:
                print(f"No log streams found. Function may not have been invoked yet.")
                return []
            
            all_events = []
            
            # Get events from recent log streams
            for stream in log_streams:
                stream_name = stream['logStreamName']
                
                try:
                    events_response = logs_client.get_log_events(
                        logGroupName=log_group,
                        logStreamName=stream_name,
                        startTime=start_ms,
                        endTime=end_ms,
                        limit=max_events
                    )
                    
                    events = events_response.get('events', [])
                    all_events.extend(events)
                    
                except ClientError:
                    continue
            
            # Sort by timestamp
            all_events.sort(key=lambda x: x['timestamp'])
            
            # Limit to max_events
            all_events = all_events[-max_events:]
            
            if not all_events:
                print(f"No log events found in the last {minutes} minutes")
                return []
            
            print(f"Found {len(all_events)} log event(s):")
            print()
            
            formatted_events = []
            for event in all_events:
                timestamp = datetime.fromtimestamp(event['timestamp'] / 1000)
                message = event['message'].rstrip()
                
                print(f"[{timestamp.strftime('%Y-%m-%d %H:%M:%S')}] {message}")
                
                formatted_events.append({
                    'timestamp': timestamp,
                    'message': message
                })
            
            return formatted_events
            
        except ClientError as e:
            error_code = e.response['Error']['Code']
            if error_code == 'ResourceNotFoundException':
                print(f"No logs found. Function '{function_name}' has not been invoked yet.")
                return []
            else:
                raise
        
    except ClientError as e:
        print(f"ERROR: Failed to retrieve logs - {e}")
        return []

# Test: Get recent logs from our function
logs = get_cloudwatch_logs('boto3-hello-world', minutes=30)

Retrieving logs for 'boto3-hello-world' (last 30 minutes)...
Log Group: /aws/lambda/boto3-hello-world
--------------------------------------------------------------------------------
Found 10 log event(s):

[2026-01-29 21:58:30] INIT_START Runtime Version: python:3.12.v102	Runtime Version ARN: arn:aws:lambda:us-east-2::runtime:e7b9763d75065b03bd2f1c073b67ce436566c24009faa1c13d7389d7ba98f259
[2026-01-29 21:58:30] START RequestId: f96a7ae4-4ba4-4604-8e00-b8b59db4a051 Version: $LATEST
[2026-01-29 21:58:30] END RequestId: f96a7ae4-4ba4-4604-8e00-b8b59db4a051
[2026-01-29 21:58:30] REPORT RequestId: f96a7ae4-4ba4-4604-8e00-b8b59db4a051	Duration: 1.99 ms	Billed Duration: 99 ms	Memory Size: 256 MB	Max Memory Used: 36 MB	Init Duration: 96.67 ms
[2026-01-29 21:58:37] START RequestId: f39b2283-f67a-4397-a2bd-d99073dfefe5 Version: $LATEST
[2026-01-29 21:58:37] END RequestId: f39b2283-f67a-4397-a2bd-d99073dfefe5
[2026-01-29 21:58:37] REPORT RequestId: f39b2283-f67a-4397-a2bd-d99073dfefe5	Duration: 

### Function 15: Get Function Metrics

Retrieve CloudWatch metrics to monitor function performance and identify issues.

In [47]:
def get_function_metrics(function_name, hours=1):
    """
    Retrieve CloudWatch metrics for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
        hours (int): Look back this many hours (default: 1)
    
    Returns:
        dict: Dictionary with metrics (invocations, errors, duration, throttles)
    
    Key metrics for data engineers:
        - Invocations: How many times function ran
        - Errors: Number of failed executions
        - Duration: Average execution time
        - Throttles: Times function was rate-limited
        - ConcurrentExecutions: Number of parallel runs
    """
    try:
        # Create CloudWatch client
        cloudwatch = boto3.client('cloudwatch',
                                  region_name=AWS_REGION,
                                  aws_access_key_id=ACCESS_KEY,
                                  aws_secret_access_key=SECRET_KEY)
        
        print(f"Retrieving metrics for '{function_name}' (last {hours} hour(s))...")
        print("-" * 80)
        
        # Calculate time range
        end_time = datetime.now()
        start_time = end_time - timedelta(hours=hours)
        
        # Metrics to retrieve
        metrics_to_get = [
            'Invocations',
            'Errors', 
            'Duration',
            'Throttles',
            'ConcurrentExecutions'
        ]
        
        results = {}
        
        for metric_name in metrics_to_get:
            response = cloudwatch.get_metric_statistics(
                Namespace='AWS/Lambda',
                MetricName=metric_name,
                Dimensions=[
                    {
                        'Name': 'FunctionName',
                        'Value': function_name
                    }
                ],
                StartTime=start_time,
                EndTime=end_time,
                Period=3600,  # 1 hour periods
                Statistics=['Sum', 'Average', 'Maximum'] if metric_name == 'Duration' else ['Sum']
            )
            
            datapoints = response.get('Datapoints', [])
            
            if datapoints:
                if metric_name == 'Duration':
                    # Duration is in milliseconds
                    avg_duration = sum(dp.get('Average', 0) for dp in datapoints) / len(datapoints)
                    max_duration = max(dp.get('Maximum', 0) for dp in datapoints)
                    results[metric_name] = {
                        'average_ms': round(avg_duration, 2),
                        'max_ms': round(max_duration, 2)
                    }
                else:
                    total = sum(dp.get('Sum', 0) for dp in datapoints)
                    results[metric_name] = int(total)
            else:
                results[metric_name] = 0 if metric_name != 'Duration' else {'average_ms': 0, 'max_ms': 0}
        
        # Display results
        print(f"Invocations: {results.get('Invocations', 0)}")
        print(f"Errors: {results.get('Errors', 0)}")
        
        if results.get('Invocations', 0) > 0:
            error_rate = (results.get('Errors', 0) / results.get('Invocations', 1)) * 100
            print(f"Error Rate: {error_rate:.2f}%")
        
        duration_data = results.get('Duration', {})
        print(f"Average Duration: {duration_data.get('average_ms', 0)} ms")
        print(f"Max Duration: {duration_data.get('max_ms', 0)} ms")
        print(f"Throttles: {results.get('Throttles', 0)}")
        print(f"Concurrent Executions: {results.get('ConcurrentExecutions', 0)}")
        
        # Cost estimation (approximate)
        if results.get('Invocations', 0) > 0:
            # Get function memory configuration
            func_config = lambda_client.get_function_configuration(FunctionName=function_name)
            memory_mb = func_config['MemorySize']
            
            avg_duration_sec = duration_data.get('average_ms', 0) / 1000
            gb_seconds = (memory_mb / 1024) * avg_duration_sec * results['Invocations']
            
            # AWS pricing: $0.0000166667 per GB-second (approximate)
            estimated_cost = gb_seconds * 0.0000166667
            
            print(f"Estimated Cost: ${estimated_cost:.4f}")
        
        print("-" * 80)
        print(f"SUCCESS: Retrieved metrics for '{function_name}'")
        
        return results
        
    except ClientError as e:
        print(f"ERROR: Failed to retrieve metrics - {e}")
        return {}

# Test: Get metrics from our function
metrics = get_function_metrics('boto3-hello-world', hours=2)

Retrieving metrics for 'boto3-hello-world' (last 2 hour(s))...
--------------------------------------------------------------------------------
Invocations: 0
Errors: 0
Average Duration: 0 ms
Max Duration: 0 ms
Throttles: 0
Concurrent Executions: 0
--------------------------------------------------------------------------------
SUCCESS: Retrieved metrics for 'boto3-hello-world'
